# 0. 프로젝트 환경구축 하기

## Colab과 Github 활용하기

### Colab에 drive 마운트 하기
- 자신의 구글 계정 drive에 접근합니다. 
- 구글에서 인증요청이 오면 동의하고 진행합니다.
- 순서대로 진행하게 되면 좌측 카테고리에 drive 폴더가 생성됩니다. 
- 생성되지 않았다면 새로고침 버튼을 통해 drive 폴더가 표시되게 합니다. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

- 드라이브의 MyDrive 폴더로 현재 위치를 변경합니다.
- 이곳에 우리가 작업할 프로젝트를 불러올 것입니다. 

In [2]:
cd "/content/drive/MyDrive"

### Github에서 프로젝트 불러오기

- Github를 통해서 실습 프로젝트를 불러옵니다. 

In [3]:
!git clone https://github.com/CTGAN-syntheticdata-generation/Selection-of-criminals_with_syndata.git

- 실습데이터가 있는 폴더로 현재 위치를 변경합니다. 

In [4]:
cd "/content/drive/MyDrive/Selection-of-criminals_with_syndata/CTGAN_syndata_generaton" 

# 1. Generate synthetic data with CTGAN

## CTGAN에 대한 소개

- CTGAN은 Conditional Tabular GAN의 약자이며 테이블 형식 데이터를 생성하기 위해 특별히 설계된 GAN 아키텍처 유형입니다. 
- CTGAN은 원본 데이터의 통계적 특성을 유지하면서 실제 데이터와 유사한 표 형식 데이터의 새 행을 생성할 수 있습니다.
- CTGAN은 실제 데이터와 합성 데이터를 구분하는 판별자와 실제 데이터 분포와 일치하는 합성 데이터를 생성하는 생성기를 사용하여 실제 데이터와 합성 데이터 모두에 대해 모델을 교육하여 작동합니다. 
- 생성기는 실제 데이터와 유사하고 사실적인 합성 데이터를 생성하도록 훈련되는 반면 판별자는 실제 데이터와 합성 데이터를 구별하도록 훈련됩니다.

## Load Library

- CTGAN 라이브러리 설치하기
- 최초 실행시 사용자 환경에 따라 오류가 발생할 수 있으나 다시 실행하면 일부 문제 해결 가능

In [5]:
!pip install ctgan

- Library for data type
- 데이터 프레임을 사용하기 위해 판다스 라이브러리를 활용합니다. 

In [6]:
import pandas as pd

- Library for model trainning 
- Use torch to facilitate computation using GPU

In [7]:
import torch
#Tools for Python Memory Management
import gc 
from ctgan import CTGAN

C:\Users\0614_\anaconda3\envs\wco_ctgan\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Check we can use GPU(with cuda)

In [8]:
torch.cuda.is_available()

True

In [9]:
# Manually turn off objects that are not currently needed
gc.collect()
# Function to clear the CUDA memory cache curently in use
torch.cuda.empty_cache()

- Library for graphing
- seaborn과 matplotlib를 통해서 데이터를 시각화 합니다. 

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

- setting etc 
- copy는 데이터를 복사하는 라이브러리
- time은 시스템의 시간을 확인할 수 있는 라이브러리

In [11]:
import copy
import time

import warnings
warnings.filterwarnings("ignore")

- setting Jupyter Notebook

In [12]:
#Jupiter Cell Full Screen View
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# useful for debuging (print the results of both formulas and functions entered in one cell of Jupiter)
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
# Each column width at maximum (print all column contents)
pd.set_option('display.max_colwidth', -1)
# Show up to 500 rows
pd.set_option('display.max_rows', 500)
# Display up to 500 columns
pd.set_option('display.max_columns', 500)
# Total length of data frame
pd.set_option('display.width', 1000)

print('ready to run')
# logging starttime 
startTime = time.time()

ready to run


## Load Data

- 미리 준비해둔 가상의 관세청 데이터를 불러온다.

In [13]:
df_raw=pd.read_csv('df_syn_en.csv', encoding='utf-8-sig')

FileNotFoundError: [Errno 2] No such file or directory: 'df_syn_en.csv'

## Data copy

- 불러온 데이터를 안전하게 사용하기 위해 복사함 

In [ ]:
df_raw_copy = copy.deepcopy(df_raw)

## Check the data
- 'dec_num' deduplication (unique value)
- Sorting by 'dec_date' (ASC)
- Check for outliers ('imp_dec_code')

In [ ]:
#Change of 'dec_date' type
df_raw_copy['dec_date'] = pd.to_datetime(df_raw_copy['dec_date']) 
#'dec_num' deduplication
df_org = df_raw_copy.drop_duplicates(['dec_num'], keep = 'first') 
#'dec_date' sorting ASC
df_org = df_org.sort_values(by = ['dec_date'], axis = 0) 

In [ ]:
#불러온 데이터의 형태를 확인
df_org.shape

In [ ]:
#불러온 데이터의 예시를 확인
df_org.head()

- 불러온 데이터의 기본 정보를 확인 
- 일부 데이터가 적은 것을 확인할 수 있음

In [ ]:
#dec_date의 타입이 유일하게 다른 datetime64[ns]임을 확인
df_org.info()

- describe을 활용해 모든 칼럼에 대한 데이터 요약을 확인
- 수치형 데이터의 mean, std 등 통계정보를 요약해줌 

In [ ]:
# 범주형 데이터의 가장 빈번하게 등장하는 변수 빈번하게 등장한 횟수 등의 정보를 요약해줌
df_org.describe(include = 'all')

### Outlier Found
- Ensure that a particular value is concentrated in one column
- Recognizes that only one value can be biased and extracted during sampling

In [ ]:
df_org['imp_dec_code'].value_counts()

### Check the correlation
- 상관도 분석은 두 변수간의 선형적인 상관관계를 분석하는 방법입니다. 
- 두 변수간에 어떤 종류의 연관성이 있는 지 파악하는데 활용합니다. 
- Heatmap을 통해서 각 'imp_dec_code' 칼럼이 다른 칼럼에 어떤 영향을 미치는 지 시각적으로 확인하고자 합니다. 

#### Working with encoding data frames as numeric data

- sklearn에서 데이터 전처리를 위해 사용하는 라이브러리 LableEncoder를 활용
- LabelEncoder는 범주형 데이터에 라벨링을 통해서 수치형 변수로 변환합니다.

In [1]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

#복사한 데이터를 순서대로 가져와 encoding 후에 데이터를 입력합니다. 
for column_name,item in df_raw_copy.iteritems(): 
    encoder.fit(item)
    labels = encoder.transform(item)
    df_raw_copy[column_name] = labels

NameError: name 'df_raw_copy' is not defined

#### Making Heatmap

In [ ]:
#matplotlib의 font_manager 객체를 통해 우리가 그릴 Heatmap의 폰트를 설정합니다.
import matplotlib.font_manager as fm

#구글 코랩에 기본으로 제공되는 DejaVu Sans라는 글꼴을 활용합니다. 
plt.rcParams['font.family'] = 'DejaVu Sans'

#font 설정을 위한 함수, 폰트가 설치되어있는 경로와 사이즈를 결정한다. 
def set_font():
    font_path = '/usr/local/lib/python3.7/dist-packages/matplotlib/mpl-data/fonts/ttf/'
    fontprop = fm.FontProperties(fname=font_path, size=12)
    return fontprop

fontprop = set_font()

In [ ]:
#데이터프레임의 상관도를 구합니다. 
corr = df_raw_copy.corr()

- Heatmap을 확인해 보니 imp_dec_code와 높은 상관도를 가지는 칼럼은 존재하지 않습니다. 
- 특별한 조치를 취하지 않고 그대로 작업을 진행하겠습니다. 

In [ ]:
sns.heatmap(corr, cmap='coolwarm', annot=False, linewidths =0.5, annot_kws={'size':10, 'fontproperties':fontprop},vmin=-1.0)

## Random data sampling 
- 기본 데이터에서 훈련시킬 데이터를 추출합니다. 
- 데이터의 양은 1500개 정도가 적당합니다. 샘플 양이 많아지면 작업시간이 크게 증가합니다.

In [ ]:
df_sample=df_org.sample(1500,replace=False)

## 샘플 데이터 처리

- Preserve correlation by grouping well-correlated columns into a single multiple column
- Correlation may not be preserved during data generation

In [ ]:
cols = ['HS10', 'country_ship_code', 'country_orig_code', 'trff_rate', 'trff_class_code', 'dec_weight','taxabal_price_KRW','crime_yn', 'key_exposure']
df_sample['HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key'] = df_sample[cols].apply(lambda row: '^'.join(row.values.astype(str)), axis=1)

### 불필요한 칼럼 삭제
- Delete Duplicate Columns
- 'dec_date'의 데이터 타입은 일부 colab 환경에서 오류가 발생하여 삭제하고 진행하겠습니다. (학습에는 큰 지장을 주지 않습니다.)

In [ ]:
df_sample=df_sample.drop(cols, axis=1)
df_sample=df_sample.drop('dec_date', axis=1)

## Trainning model with CTGAN 

- CTGAN 알고리즘을 활용하여 모델 학습을 실시합니다. 

In [ ]:
# 학습할 카테고리의 리스트를 변수로 선언합니다. 
categorical_columns =[ 'dec_num', 'dec_custom_code','imp_dec_code','imp_trd_code',\
                      'imp_typ_code','collect_code', 'typ_transport_code','dec_mark','importer',\
                      'ovs_cust_code','exps_carr_code','country_orig_mark_code','HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key']

In [ ]:
#CTGAN 모델을 학습합니다. 
ctgan = CTGAN(verbose=True)
startTime = time.time()
ctgan.fit(df_sample, categorical_columns, epochs = 500)
endTime = time.time()

print('time:', endTime - startTime )

## Generating synthetic data with trained model

- 샘플링 한 데이터 갯수 만큼 데이터를 생성함
- 더 많은 데이터를 생성할 경우 상관도를 비롯해 두 테이블간 통계적 유사함을 점차 잃게 된다. 

In [ ]:
#샘플링 한 데이터 갯수 만큼 데이터를 생성함
df_syn = ctgan.sample(len(df_sample))

In [ ]:
#신고번호를 기준으로 정렬함
df_syn = df_syn.sort_values('dec_num')

## Save the new data

In [ ]:
# 합성된 데이터의 정보를 확인 합니다. 
df_syn.info()

In [ ]:
#여러 칼럼을 합쳐두었던 데이터를 분리하여 각 칼럼을 생성합니다. 
df_syn['HS10']=df_syn.HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key.str.split('^').str[0]
df_syn['country_ship_code']=df_syn.HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key.str.split('^').str[1]
df_syn['country_orig_code']=df_syn.HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key.str.split('^').str[2]
df_syn['trff_rate']=df_syn.HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key.str.split('^').str[3]
df_syn['trff_class_code']=df_syn.HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key.str.split('^').str[4]
df_syn['dec_weight']=df_syn.HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key.str.split('^').str[5]
df_syn['taxabal_price_KRW']=df_syn.HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key.str.split('^').str[6]
df_syn['crime_yn']=df_syn.HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key.str.split('^').str[7]
df_syn['key_exposure']=df_syn.HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key.str.split('^').str[8]

In [ ]:
#여러 칼럼이 합쳐진 칼럼을 삭제 합니다.
df_syn = df_syn.drop(['HS10_ship_orig_tffrt_tffcd_weight_taxprice_crime_key'],axis=1)

- Sort columns the same as the original data
- 여러 칼럼을 합쳤다가 분리가 되었기 때문에 칼럼의 순서가 뒤섞였습니다. 

In [2]:
#기존 데이터의 칼럼 순서대로 정렬합니다.
df_tmp = df_syn[['HS10','country_ship_code','country_orig_code','trff_rate',\
                 'trff_class_code','country_orig_mark_code','dec_weight','taxabal_price_KRW','crime_yn','key_exposure']]
df_syn = df_syn.drop(['HS10','country_ship_code','country_orig_code','trff_rate',\
                 'trff_class_code','country_orig_mark_code','dec_weight','taxabal_price_KRW','crime_yn','key_exposure'],axis=1)
df_syn = pd.concat([df_syn, df_tmp], axis = 1)

NameError: name 'df_syn' is not defined

In [ ]:
# 합성데이터의 정보를 점검합니다.
df_syn.info()

In [ ]:
#데이터를 지정된 경로에 저장합니다. 
df_syn.to_csv('./data_sample/df_syn_en_14.csv', index=False, encoding='utf-8-sig')